라이브러리

In [1]:
import os
import re
import torch
import pandas as pd
import numpy as np

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

데이터 준비 및 정제

In [3]:
# word_dict.csv load
df = pd.read_csv("./word_dict.csv")
df.head()

,Unnamed: 0,Word,Content
0,0,가계부실위험지수(HDRI),\n가구의 소득 흐름은 물론 금융 및 실물 자산까지 종합적으로 고려하여 가계부채의\...
1,1,가계수지,\n가정에서 일정 기간의 수입(명목소득)과 지출을 비교해서 남았는지 모자랐는지를\n...
2,2,가계순저축률,\n일반적으로 저축률은 저축액을 처분가능소득으로 나눈 비율을 말한다. 마찬가지로\n...
3,3,가계신용통계,\n가계신용통계는 가계부문에 대한 신용공급 규모를 나타내는 통계이다. 가계신용은\n...
4,4,가계처분가능소득,\n가계처분가능소득(PDI; Personal Disposable Income)은 가...


In [4]:
# drop "Unnamed:0"
df = df.drop('Unnamed: 0', axis=1)
df.head()

,Word,Content
0,가계부실위험지수(HDRI),\n가구의 소득 흐름은 물론 금융 및 실물 자산까지 종합적으로 고려하여 가계부채의\...
1,가계수지,\n가정에서 일정 기간의 수입(명목소득)과 지출을 비교해서 남았는지 모자랐는지를\n...
2,가계순저축률,\n일반적으로 저축률은 저축액을 처분가능소득으로 나눈 비율을 말한다. 마찬가지로\n...
3,가계신용통계,\n가계신용통계는 가계부문에 대한 신용공급 규모를 나타내는 통계이다. 가계신용은\n...
4,가계처분가능소득,\n가계처분가능소득(PDI; Personal Disposable Income)은 가...


In [5]:
# clean text
def clean_text(text):
    text = re.sub(r"\n+", " ", text)  # 연속된 줄바꿈은 공백으로 대체
    text = re.sub(r"-{2,}"," ", text)  # ----- 제거
    text = re.sub(r"\s{2,}", " ", text)  # 중복된 공백 제거
    text = text.strip()  # 양쪽 공백 제거
    return text

df['Content'] = df['Content'].apply(clean_text)
df.to_csv("cleaned_word_dict.csv", index=False, encoding="utf-8-sig")

csv 파일 로드

In [6]:
# cleaned_word_dict.csv 파일 확인
df = pd.read_csv("./cleaned_word_dict.csv")
df.head()

,Word,Content
0,가계부실위험지수(HDRI),가구의 소득 흐름은 물론 금융 및 실물 자산까지 종합적으로 고려하여 가계부채의 부실...
1,가계수지,가정에서 일정 기간의 수입(명목소득)과 지출을 비교해서 남았는지 모자랐는지를 표시한...
2,가계순저축률,일반적으로 저축률은 저축액을 처분가능소득으로 나눈 비율을 말한다. 마찬가지로 가계순...
3,가계신용통계,가계신용통계는 가계부문에 대한 신용공급 규모를 나타내는 통계이다. 가계신용은 금융기...
4,가계처분가능소득,가계처분가능소득(PDI; Personal Disposable Income)은 가계가...


In [3]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="./cleaned_word_dict.csv")
docs = loader.load()
print(f"문서의 수: {len(docs)}")

print(f"\n[페이지내용]\n{docs[10].page_content[:500]}")
print(f"\n[metadata]\n{docs[10].metadata}\n")

문서의 수: 700

[페이지내용]
﻿Word: 가상통화공개(ICO)
Content: 가상통화(ICO; Initial Coin Offering) 공개는 주로 혁신적인 신생기업(startup)이 암호 화화폐(cryptocurrency) 또는 디지털 토큰(digital token, 일종의 투자증명)을 이용하여 자금을 조달할 수 있는 크라우드펀딩(crowd funding)의 한 방식이다. 가상통화공개 (ICO)에서 새로 발행된 암호화화폐는 법화(legal tender) 또는 비트코인 등 기존의 가상 통화와 교환되어 투자자에게 팔린다. 이 용어는 거래소에 상장하려는 기업이 투자자에게 자기 주식을 처음 공개적으로 매도하는 기업공개(IPO; Initial Public Offering)에서 연유 되었다고 볼 수 있다. 기업공개(IPO)에 참여한 투자자는 해당 기업의 소유권과 관련하여 주식을 획득한다. 반면 가상통화공개(ICO)에 참여한 투자자는 해당 신생기업의 코인 (coins) 또는 토큰을 얻는데, 이는 해당 기업이 제안한 프로

[metadata]
{'source': './cleaned_word_dict.csv', 'row': 10}



데이터 분할

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"청크 수: {len(split_documents)}")

청크 수: 721


임베딩 & 벡터스토어 생성

In [5]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

In [13]:
# generate FAISS VectorStore
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
vectorstore = FAISS.from_documents(
    documents=split_documents, embedding=embedding_model
)

vectorstore.save_local("faiss_index")

In [6]:
# load FAISS VectorStore
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
vectorstore = FAISS.load_local("faiss_index", embedding_model, allow_dangerous_deserialization=True)

C:\Users\lgpc\AppData\Local\Temp\ipykernel_11452\1604933528.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")


Retriever 생성 & LLM 모델 불러오기

In [7]:
from huggingface_hub import login
hf_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")
login(token=hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\lgpc\.cache\huggingface\token
Login successful


In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain.llms import HuggingFacePipeline, HuggingFaceHub

# define retriever
retriever = vectorstore.as_retriever(search_kwargs={"k":5})

# load LLaMA
model_name = "meta-llama/Llama-3.2-1B" 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# generate Hugging Face Pipeline
from transformers import pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    #max_length=1024,
    max_new_tokens=300,
    temperature=0.7,
    top_p=0.9,
)

llm = HuggingFacePipeline(pipeline=pipe)

C:\Users\lgpc\AppData\Local\Temp\ipykernel_11452\3925770001.py:24: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [9]:
template = """
You are a helpful assistant specialized in explaining financial terms.
Use the context below to answer the query in simple terms, provide examples, and any related information.

Context:
{context}

Question:
{term}

Answer:
"""

In [10]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import RetrievalQA

prompt = PromptTemplate(
    input_variables=["context", "query"], 
    template=template
)

# generate Chain
chain = (
    {"context":retriever, "term":RunnablePassthrough()}
    | prompt
    | llm 
    | StrOutputParser()
)

In [11]:
term = "가계수지"
response = chain.invoke(term)
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



You are a helpful assistant specialized in explaining financial terms.
Use the context below to answer the query in simple terms, provide examples, and any related information.

Context:
[Document(id='fca64cb1-a35f-44dc-be08-a9a74da0b842', metadata={'source': './cleaned_word_dict.csv', 'row': 1}, page_content="\ufeffWord: 가계수지\nContent: 가정에서 일정 기간의 수입(명목소득)과 지출을 비교해서 남았는지 모자랐는지를 표시한 것을 가계수지(household's total income and expenditure)라 한다. 가계수지가 흑자를 냈다면 그 가정은 벌어들인 수입 일부만을 사용했다는 것을 의미하며, 적자를 냈다면 수입 외에 빚을 추가로 얻어 사용한 것이라고 보아야 한다. 우리나라는 통계청에 서 가계의 수입과 지출을 조사하여 국민의 소득수준 및 생활실태를 파악하기 위해 표본으로 선정된 가계에 가계부를 나누어 주고 한 달간의 소득과 지출을 기록하도록 한 다음 이를 토대로 가계수지 통계를 작성하여 발표하고 있다. 가계부의 소득항목에는 근로소득･사업소득･재산소득･이전소득 항목이 있고, 비용항목에는 식료품비･주거비･ 수도광열비･보건의료비･교육비 항목이 있다. \n연관검색어 : 경상수지, 재정수지"), Document(id='d340163b-16f8-45a7-a1ea-11eb8a73820e', metadata={'source': './cleaned_word_dict.csv', 'row': 3}, page_content='\ufeffWord: 가계신용통계\nContent: 가계신용통계는 가계부문에 대한 신용공급 규모를 나타내는 통계이다. 가계신용은 금융기관뿐 아니라 정부, 판매회사 등 기타기관이 가계에 제공